In [1]:
import pandas as pd
import numpy as np
import arcpy
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import geopandas as gpd
# plot import 
import colorcet as cc
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import cmocean
import colormaps as cmaps 
import numpy as np, string, matplotlib as mpl

In [3]:
inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient"
#Huron_Lake = r"D:\Users\abolmaal\code\boundry\hydro_p_LakeHuron\hydro_p_LakeHuron.shp"
Huron_Lake = r"S:\Data\GIS_Data\Downloaded\Worldwide_Datasets\Natural_Earth_Data\10m_physical\ne_10m_lakes.shp"
Streamwatershed = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWatershed_LH_NAD1983_NASA_Invasive.shp"
landshapefile  = r"S:\Data\GIS_Data\Downloaded\Worldwide_Datasets\Natural_Earth_Data\10m_cultural\ne_10m_admin_0_countries_lakes.shp"
inTN = os.path.join(inDir, "TN_Annual_delTotal_header_kgcellday.tif")
inTP = os.path.join(inDir, "TP_Annual_delTotal_header_kgcellday.tif")

In [13]:
# -----------------------------
# Map settings & units
# -----------------------------
CELL_SIZE_X = 119.986262
CELL_SIZE_Y = 119.977553
X_MIN, X_MAX = 0.75e6, 1.30e6
Y_MIN, Y_MAX = 0.63e6, 1.20e6

probs = [0.00, 0.25, 0.50, 0.75, 1.00]
font_size = 8
mpl.rcParams.update({'font.size': font_size})

# How to handle zeros/negatives
EXCLUDE_NONPOSITIVE = True        # don't include <=0 in quantiles
HIDE_NONPOSITIVE_ON_MAP = True    # make <=0 transparent

# Unit conversion kg/cell/day → kg/ha/yr
cell_area_ha = (CELL_SIZE_X * CELL_SIZE_Y) / 10_000.0
KG_CELL_DAY_to_KG_HA_YR = 365.0 / cell_area_ha

# -----------------------------
# Helpers
# -----------------------------
def add_panel_labels(axes, labels=None, dx=0.02, dy=0.98, fsize=None):
    """
    Add 'a)', 'b)', ... (or custom labels) at top-left of each axes.
    Works for single Axes, 1D list, or 2D array of axes.
    """
    axs = np.ravel(np.array(axes))  # flatten in case it's 2D
    if labels is None:
        labels = [f"{ch})" for ch in string.ascii_lowercase]
    if fsize is None:
        fsize = mpl.rcParams.get("font.size", 10) + 6

    for i, ax in enumerate(axs):
        lab = labels[i] if i < len(labels) else f"{string.ascii_lowercase[i]})"
        ax.text(
            dx, dy, lab,
            transform=ax.transAxes, ha="left", va="top",
            fontsize=fsize, fontweight="bold",
            bbox=dict(facecolor="none", edgecolor="none", alpha=0.7, pad=0.2),
            zorder=1000, clip_on=False
        )
        
def raster_extent(transform, h, w):
    left, top = transform.c, transform.f
    px_w, px_h = transform.a, transform.e
    return (left, left + w*px_w, top + h*px_h, top)

def to_kg_ha_yr(arr, nodata):
    a = arr.astype(float)
    if nodata is not None:
        a = np.where(arr == nodata, np.nan, a)
    return a * KG_CELL_DAY_to_KG_HA_YR

def quantile_breaks_positive(values, q_probs, exclude_nonpositive=True):
    v = values[np.isfinite(values)]
    if exclude_nonpositive:
        v = v[v > 0]
    if v.size == 0:
        raise ValueError("No positive finite values available to compute quantiles.")
    b = np.quantile(v, q_probs).astype(float)
    # ensure strictly increasing
    for i in range(1, len(b)):
        if b[i] <= b[i-1]:
            b[i] = np.nextafter(b[i-1], np.inf)
    return b

def boundary_norm(breaks):  # discrete bins
    return mpl.colors.BoundaryNorm(breaks, ncolors=len(breaks)-1, clip=True)

def sci_str(x: float, sig: int = 3) -> str:
    """Format a value as a×10^n using Matplotlib mathtext (handles tiny values)."""
    if not np.isfinite(x) or x == 0:
        return "0"
    e = int(np.floor(np.log10(abs(x))))
    m = x / (10.0 ** e)
    return rf"{m:.{sig}g} $\times 10^{{{e}}}$"

def sci_range_labels_individual(breaks, sig: int = 3):
    """Labels like 'a×10^n – b×10^m' for each bin (no shared exponent)."""
    b = np.asarray(breaks, float)
    labels = [f"{sci_str(lo, sig)} – {sci_str(hi, sig)}" for lo, hi in zip(b[:-1], b[1:])]
    return labels

def plot_one(ax, arr_kg_ha_yr, transform, brks, lake_gdf, ws_gdf, land_gdf, title):
    # Optionally hide non-positive pixels
    data = arr_kg_ha_yr.copy()
    if HIDE_NONPOSITIVE_ON_MAP:
        data = np.where(np.isfinite(data) & (data <= 0), np.nan, data)

    h, w = data.shape
    extent = raster_extent(transform, h, w)

    # Discrete colormap (ColorCET)
    #cmap_disc = mpl.colors.ListedColormap(cc.cm["CET_L17"](np.linspace(0, 1, len(brks)-1)))
    cmap_disc = mpl.colors.ListedColormap(cmaps.tealrose(np.linspace(0, 1, len(brks)-1)))
    norm = boundary_norm(brks)

    # Land underlay
    if land_gdf is not None and not land_gdf.empty:
        land_gdf.plot(ax=ax, facecolor="gainsboro", edgecolor="dimgray", linewidth=0.4, zorder=0)

    # Raster
    im = ax.imshow(data, extent=extent, origin="upper", cmap=cmap_disc, norm=norm, zorder=3)

    # Overlays
    if lake_gdf is not None and not lake_gdf.empty:
        lake_gdf.plot(ax=ax, facecolor="darkgray", edgecolor="#5b7aa6", linewidth=0.3, zorder=6)

    # Streamwatershed as a black boundary (no fill)
    if ws_gdf is not None and not ws_gdf.empty:
        # For many polygons, boundary-only is cleaner than facecolor="none"
        ws_gdf.boundary.plot(ax=ax, color="black", linewidth=0.6, zorder=8)

    # Axes & extent
    ax.set_aspect("equal")
    ax.set_xlim(X_MIN, X_MAX)
    ax.set_ylim(Y_MIN, Y_MAX)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_xlabel(""); ax.set_ylabel("")
    # ax.set_title(title)  # uncomment if you want titles

    # Colorbar with per-bin scientific labels
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.02)

    mids = 0.5 * (np.array(brks[:-1]) + np.array(brks[1:]))
    cbar.set_ticks(mids)
    cbar.set_ticklabels(sci_range_labels_individual(brks, sig=3))

    # 🔹 Make the numbers (tick labels) bigger
    cbar.ax.tick_params(labelsize=font_size + 4)

    # 🔹 Make the colorbar label bigger too (optional)
    cbar.ax.set_ylabel(
        r"kg ha$^{-1}$ yr$^{-1}$", rotation=90, fontsize=font_size + 2
    )


    # Optional single legend entry for boundary
    # legend_line = Line2D([0], [0], color="black", lw=0.8, label="Stream watershed boundary")
    # ax.legend(handles=[legend_line], loc="lower right",
    #           frameon=True, framealpha=0.9, fontsize=font_size)

# -----------------------------
# Read rasters (native CRS)
# -----------------------------
with rasterio.open(inTN) as src_tn:
    arr_tn = src_tn.read(1)
    tr_tn = src_tn.transform
    crs_tn = src_tn.crs
    nd_tn = src_tn.nodata

with rasterio.open(inTP) as src_tp:
    arr_tp = src_tp.read(1)
    tr_tp = src_tp.transform
    crs_tp = src_tp.crs
    nd_tp = src_tp.nodata

# Convert to kg/ha/yr
tn_kg_ha_yr = to_kg_ha_yr(arr_tn, nd_tn)
tp_kg_ha_yr = to_kg_ha_yr(arr_tp, nd_tp)

# -----------------------------
# Vectors → raster CRS
# -----------------------------
lake = gpd.read_file(Huron_Lake).to_crs(crs_tn)
ws   = gpd.read_file(Streamwatershed).to_crs(crs_tn)
land = gpd.read_file(landshapefile).to_crs(crs_tn)

# (Optional) If you want a single outer outline, dissolve once:
# ws = ws.dissolve()             # merge polygons
# ws = ws.boundary.to_frame("geometry")  # keep only boundary geometry in a gdf

# -----------------------------
# Quantile breaks on **positive** values only
# -----------------------------
breaks_tn = quantile_breaks_positive(tn_kg_ha_yr, probs, exclude_nonpositive=EXCLUDE_NONPOSITIVE)
breaks_tp = quantile_breaks_positive(tp_kg_ha_yr, probs, exclude_nonpositive=EXCLUDE_NONPOSITIVE)

# (Optional) Shared breaks across both layers:
# all_pos = np.concatenate([
#     tn_kg_ha_yr[np.isfinite(tn_kg_ha_yr) & (tn_kg_ha_yr > 0)],
#     tp_kg_ha_yr[np.isfinite(tp_kg_ha_yr) & (tp_kg_ha_yr > 0)]
# ])
# shared = quantile_breaks_positive(all_pos, probs, exclude_nonpositive=False)
# breaks_tn = breaks_tp = shared

# -----------------------------
# Plot
# -----------------------------
dpi = 300

width_in  = 3508 / dpi   # ≈ 11.69 inches
height_in = 2481 / dpi   # ≈ 8.27 inches

fig, axes = plt.subplots(1, 2, figsize=(width_in, height_in), constrained_layout=True)
plot_one(axes[0], tn_kg_ha_yr, tr_tn, breaks_tn, lake, ws, land, "TN (Annual ΔTotal)")
plot_one(axes[1], tp_kg_ha_yr, tr_tp, breaks_tp, lake, ws, land, "TP (Annual ΔTotal)")
add_panel_labels(axes, labels=["a)", "b)"], dx=0.02, dy=0.98, fsize=font_size+4)
# -----------------------------
# Save
# -----------------------------
Figure_path = r"S:\Users\Samin\Figures\chapter1\senseflux"
os.makedirs(Figure_path, exist_ok=True)
out_png = os.path.join(Figure_path, "TN_TP_quartiles_ws_outline.jpg")
fig.savefig(out_png, dpi=dpi, bbox_inches="tight")
print(f"Saved figure → {out_png}")
plt.close(fig)

Saved figure → S:\Users\Samin\Figures\chapter1\senseflux\TN_TP_quartiles_ws_outline.jpg


In [ ]:
#bcd7ff